# Single host networking

Viri:
- [Networking overview](https://docs.docker.com/network/)

One of the reasons Docker containers and services are so powerful is that you can connect them together, or connect them to non-Docker workloads. Docker containers and services do not even need to be aware that they are deployed on Docker, or whether their peers are also Docker workloads or not. Whether your Docker hosts run Linux, Windows, or a mix of the two, you can use Docker to manage them in a platform-agnostic way.

## Docker container networking

Docker abstracts the underlying host-attached network from containers. Doing so provides
a degree of runtime environment agnosticism for the application, and allows infrastructure
managers to adapt the implementation to suit the operating environment.

**A container attached to a Docker network will get a unique IP address that is routable
from other containers attached to the same Docker network.**

The main problem with this approach is that there is no easy way for any software
running inside a container to determine the IP address of the host where the container
is running. This inhibits a container from advertising its service endpoint to
other services outside the container network.

Docker also treats networks as first-class entities. This means that they have their
own life cycle and are not bound to any other objects. You can define and manage
them directly by using the `docker network` subcommands.

To get started with networks in Docker, examine the default networks that are
available with every Docker installation. Running `docker network ls` will print a table
of all the networks to the terminal.

    NETWORK ID          NAME                             DRIVER              SCOPE
    b32b91910ef6        bridge                           bridge              local
    203a7a3fe7a1        host                             host                local
    9ee3d1826da9        none                             null                local

By default, Docker includes three networks, and each is provided by a different driver.
- The network named **bridge** is the default network and provided by a bridge driver. The
bridge driver provides intercontainer connectivity for all containers running on the
same machine. 
- The **host** network is provided by the host driver, which instructs Docker
not to create any special networking namespace or resources for attached containers.
Containers on the host network interact with the host’s network stack like uncontained
processes. 
- Finally, the **none** network uses the null driver. Containers attached to the
none network will not have any network connectivity outside themselves.

The scope of a network can take three values: local, global, or swarm. This indicates
whether the network is constrained to the machine where the network exists
(local), should be created on every node in a cluster but not route between them
(global), or seamlessly spans all of the hosts participating in a Docker swarm (multihost
or cluster-wide). As you can see, all of the default networks have the local
scope, and will not be able to directly route traffic between containers running on
different machines.

The default bridge network maintains compatibility with legacy Docker and cannot
take advantage of modern Docker features including service discovery or load balancing.
Using it is not recommended. So the first thing you should do is create your
own bridge network.

### Network drivers

Docker’s networking subsystem is pluggable, using drivers. Several drivers exist by default, and provide core networking functionality:
- **bridge**: The default network driver. If you don’t specify a driver, this is the type of network you are creating. Bridge networks are usually used when your applications run in standalone containers that need to communicate. *User-defined bridge networks are best when you need multiple containers to communicate on the same Docker host.*
- **host**: For standalone containers, remove network isolation between the container and the Docker host, and use the host’s networking directly. host is only available for swarm services on Docker 17.06 and higher. *Host networks are best when the network stack should not be isolated from the Docker host, but you want other aspects of the container to be isolated.*
- **overlay**: Overlay networks connect multiple Docker daemons together and enable swarm services to communicate with each other. You can also use overlay networks to facilitate communication between a swarm service and a standalone container, or between two standalone containers on different Docker daemons. This strategy removes the need to do OS-level routing between these containers. *Overlay networks are best when you need containers running on different Docker hosts to communicate, or when multiple applications work together using swarm services.*
- **macvlan**: Macvlan networks allow you to assign a MAC address to a container, making it appear as a physical device on your network. The Docker daemon routes traffic to containers by their MAC addresses. Using the macvlan driver is sometimes the best choice when dealing with legacy applications that expect to be directly connected to the physical network, rather than routed through the Docker host’s network stack.
- **none**: For this container, disable all networking. Usually used in conjunction with a custom network driver. none is not available for swarm services. 
- **Network plugins**: You can install and use third-party network plugins with Docker. These plugins are available from Docker Hub or from third-party vendors. See the vendor’s documentation for installing and using a given network plugin.

## Bridge networks

In terms of networking, a bridge network is a Link Layer device which forwards traffic between network segments. A bridge can be a hardware device or a software device running within a host machine’s kernel.

> The Docker bridge network driver uses Linux namespaces, virtual Ethernet devices,
and the Linux firewall to build a specific and customizable virtual network topology
called a bridge.

In terms of Docker, a bridge network uses a software bridge which allows containers connected to the same bridge network to communicate, while providing isolation from containers which are not connected to that bridge network. The Docker bridge driver automatically installs rules in the host machine so that containers on different bridge networks cannot communicate directly with each other.

> Containers have their own private loopback interface and a separate virtual Ethernet
interface linked to another virtual interface in the host’s namespace. These two
linked interfaces form a link between the host’s network and the container.

**Bridge networks apply to containers running on the same Docker daemon host.**

When you start Docker, a default bridge network (also called bridge) is created automatically, and newly-started containers connect to it unless otherwise specified. You can also create user-defined custom bridge networks. User-defined bridge networks are superior to the default bridge network.

Just like
typical home networks, each container is assigned a unique private IP address that’s
not directly reachable from the external network.

Connections are routed through
another Docker network that routes traffic between containers and may connect to
the host’s network to form a bridge.

**Containers connected to the same user-defined bridge network effectively expose all ports to each other.**

### Differences between user-defined bridges and the default bridge

- **User-defined bridges provide automatic DNS resolution between containers**: Containers on the default bridge network can only access each other by IP addresses, unless you use the --link option, which is considered legacy. On a user-defined bridge network, containers can resolve each other by name or alias.
- **User-defined bridges provide better isolation**: All containers without a --network specified, are attached to the default bridge network. This can be a risk, as unrelated stacks/services/containers are then able to communicate. Using a user-defined network provides a scoped network in which only containers attached to that network are able to communicate.
- **Containers can be attached and detached from user-defined networks on the fly**: During a container’s lifetime, you can connect or disconnect it from user-defined networks on the fly. To remove a container from the default bridge network, you need to stop the container and recreate it with different network options.
- **Each user-defined network creates a configurable bridge**: User-defined bridge networks are created and configured using docker network create. If different groups of applications have different network requirements, you can configure each user-defined bridge separately, as you create it.
- **Linked containers on the default bridge network share environment variables**

### Manage a user-defined bridge

Use the docker `network create command` to create a user-defined bridge network.

    docker network create my-net
    docker network ls

You can specify the subnet, the IP address range, the gateway, and other options. See the docker network create reference or the output of docker network create --help for details.

> [docker network create](https://docs.docker.com/engine/reference/commandline/network_create/#specify-advanced-options)

Use the docker network rm command to remove a user-defined bridge network. If containers are currently connected to the network, disconnect them first.

    docker network rm my-net

    docker network create \
        --driver bridge \
        --label project=dockerinaction \
        --label chapter=5 \
        --attachable \
        --scope local \
        --subnet 10.0.42.0/24 \
        --ip-range 10.0.42.128/25 \
        user-network

This command creates a new local bridge network named user-network. Adding
label metadata to the network will help in identifying the resource later. Marking the
new network as attachable allows you to attach and detach containers to the network
at any time. Here you’ve manually specified the network scope property and set it to
the default value for this driver. Finally, a custom subnet and assignable address range
was defined for this network, 10.0.42.0/24, assigning from the upper half of the last
octet (10.0.42.128/25). This means that as you add containers to this network, they
will receive IP addresses in the range from 10.0.42.128 to 10.0.42.255.

### Exploring a bridge network

If you’re going to run network software inside a container on a container network,
you should have a solid understanding of what that network looks like from within
a container. Start exploring your new bridge network by creating a new container
attached to that network:

    docker run -it \
        --network user-network \
        --name network-explorer \
        alpine:3.8 \
        sh

Get a list of the IPv4 addresses available in the container from your terminal (which is
now attached to the running container) by running the following:

    ip -f inet -4 -o addr

The results should look something like this:

    1: lo inet 127.0.0.1/8 scope host lo\ ...
    18: eth0 inet 10.0.42.129/24 brd 10.0.42.255 scope global eth0\ ...

You can see from this list that the container has two network devices with IPv4
addresses. Those are the loopback interface (or localhost) and eth0 (a virtual Ethernet
device), which is connected to the bridge network. Further, you can see that eth0
has an IP address within the range and subnet specified by the user-network configuration
(the range from 10.0.42.128 to 10.0.42.255). That IP address is the one that any
other container on this bridge network would use to communicate with services you
run in this container. The loopback interface can be used only for communication
within the same container.

Next, create another bridge network and attach your running network-explorer
container to both networks. First, detach your terminal from the running container
(press Ctrl-P and then Ctrl-Q) and then create the second bridge network:

    docker network create \
        --driver bridge \
        --label project=dockerinaction \
        --label chapter=5 \
        --attachable \
        --scope local \
        --subnet 10.0.43.0/24 \
        --ip-range 10.0.43.128/25 \
        user-network2

Once the second network has been created, you can attach the network-explorer
container (still running):

    docker network connect user-network2 network-explorer

After the container has been attached to the second network, reattach your terminal
to continue your exploration:

    docker attach network-explorer

Now, back in the container, examining the network interface configuration again will
show something like this:

    1: lo inet 127.0.0.1/8 scope host lo\ ...
    18: eth0 inet 10.0.42.129/24 brd 10.0.42.255 scope global eth0\ ...
    20: eth1 inet 10.0.43.129/24 brd 10.0.43.255 scope global eth1\ ...

As you might expect, this output shows that the network-explorer container is
attached to both user-defined bridge networks.

Networking is all about communication between multiple parties, and examining
a network with only one running container can be a bit boring. But is there anything
else attached to a bridge network by default? Another tool is needed to
continue exploring. Install the nmap package inside your running container by using
this command:

    apk update && apk add nmap

Nmap is a powerful network inspection tool that can be used to scan network address
ranges for running machines, fingerprint those machines, and determine what services
they are running. For our purposes, we simply want to determine what other containers
or other network devices are available on our bridge network. Run the following command
to scan the 10.0.42.0/24 subnet that we defined for our bridge network:

    nmap -sn 10.0.42.* -sn 10.0.43.* -oG /dev/stdout | grep Status

This shows that only two devices are attached to each of the bridge networks: the gateway
adapters created by the bridge network driver and the currently running container.
Create another container on one of the two bridge networks for more interesting results.
Detach from the terminal again (Ctrl-P, Ctrl-Q) and start another container attached
to user-network2. Run the following:

    docker run -d \
        --name lighthouse \
        --network user-network2 \
        alpine:3.8 \
        sleep 1d

After the lighthouse container has started, reattach to your network-explorer
container:

    docker attach network-explorer

And from the shell in that container, run the network scan again. The results show
that the lighthouse container is up and running, and accessible from the networkexplorer
container via its attachment to user-network2.

Discovering the lighthouse container on the network confirms that the network
attachment works as expected, and demonstrates how the DNS-based service discovery
system works. When you scanned the network, you discovered the new node by
its IP address, and nmap was able to resolve that IP address to a name. This means
that you (or your code) can discover individual containers on the network based on
their name. Try this yourself by running nslookup lighthouse inside the container.
Container hostnames are based on the container name, or can be set manually at container
creation time by specifying the --hostname flag.

## Host and none network